# Importing the libraries
In this part, we are importing the libraries we need.

In [1]:
# Importing files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import sys
import glob
import datetime
import calendar
import csv

# Spark imports
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.functions import col, asc, desc, split

# used to check if the array has a value
from pyspark.sql.functions import array_contains

# used for the encoding of categories
from sklearn.preprocessing import MultiLabelBinarizer

# Spark 
In this project, we are using apache spark as well as Pandas.

In [2]:
# Initialize a spark session.
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [3]:
spark = init_spark() # Initializate spark

# Loading the Business dataset

In [4]:
path = 'data/yelp_academic_dataset_business.json'
df_business = spark.read.json(path)

In [5]:
df_business_filtered = df_business.filter(df_business.categories.contains('Restaurants')) # Filtering per restaurants

In [6]:
df_business_filtered.count()

52268

In [7]:
df_business_filtered = df_business_filtered[['business_id','name','categories']]

In [8]:
df_business_filtered.show(10)

+--------------------+--------------------+--------------------+
|         business_id|                name|          categories|
+--------------------+--------------------+--------------------+
|MTSW4McQd7CbVtyjq...|  St Honore Pastries|Restaurants, Food...|
|CF33F8-E6oudUQ46H...|      Sonic Drive-In|Burgers, Fast Foo...|
|k0hlBqXX-Bt0vf1op...|Tsevi's Pub And G...|Pubs, Restaurants...|
|bBDDEgkFA1Otx9Lfe...|      Sonic Drive-In|Ice Cream & Froze...|
|eEOYSgkmpB90uNA7l...|Vietnamese Food T...|Vietnamese, Food,...|
|il_Ro8jwPlHresjw9...|             Denny's|American (Traditi...|
|0bPLkL0QhhPO5kt1_...|Zio's Italian Market|Food, Delis, Ital...|
|MUTTqe8uqyMdBl186...|            Tuna Bar|Sushi Bars, Resta...|
|ROeacJQwBeh05Rqg7...|                 BAP| Korean, Restaurants|
|WKMJwqnfZKsAae75R...|Roast Coffeehouse...|Coffee & Tea, Foo...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



# Loading the Review dataset

In [9]:
# Loading the review file
path = 'data/yelp_academic_dataset_review.json'
df_review = spark.read.json(path)

In [10]:
# Print the number of rows and columns
print((df_review.count(), len(df_review.columns)))

(6990280, 9)


In [11]:
#Printing the columns' name
df_review.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [12]:
df_review = df_review[['business_id','user_id','stars']]
df_review.show(10)

+--------------------+--------------------+-----+
|         business_id|             user_id|stars|
+--------------------+--------------------+-----+
|XQfwVwDr-v0ZS3_Cb...|mh_-eMZ6K5RLWhZyI...|  3.0|
|7ATYjTIgM3jUlt4UM...|OyoGAe7OKpv6SyGZT...|  5.0|
|YjUWPpI6HXG530lwP...|8g_iMtfSiwikVnbP2...|  3.0|
|kxX2SOes4o-D3ZQBk...|_7bHUi9Uuf5__HHc_...|  5.0|
|e4Vwtrqf-wpJfwesg...|bcjbaE6dDog4jkNY9...|  4.0|
|04UD14gamNjLY0IDY...|eUta8W_HdHMXPzLBB...|  1.0|
|gmjsEdUsKpj9Xxu6p...|r3zeYsv1XFBRA4dJp...|  5.0|
|LHSTtnW3YHCeUkRDG...|yfFzsLmaWF2d4Sr0U...|  5.0|
|B5XSoSG3SfvQGtKEG...|wSTuiTk-sKNdcFypr...|  3.0|
|gebiRewfieSdtt17P...|59MxRhNVhU9MYndMk...|  3.0|
+--------------------+--------------------+-----+
only showing top 10 rows



In [13]:
# Filtering REVIEWS file by only RESTAURANTS identified with the RESTAURANT in 'categories'
df_review1 = df_review.join(df_business_filtered[['business_id']], df_review.business_id == df_business_filtered.business_id ,"left_semi")
df_review1.count()

4724471

In [14]:
# Creating an auxiliar df to filter only user with more than 50 reviews
df_count_reviews = df_review1.groupBy("user_id").count() # Creating a df with the number of reviews per user
df_count_reviews1 = df_count_reviews.filter(col("count") >= 50)  # Filtering only users with more than 50 reviews
df_count_reviews1.show(10, truncate=False)

+----------------------+-----+
|user_id               |count|
+----------------------+-----+
|IeSz60ozr1yAVIH8CX9w9w|149  |
|FlXBpK_YZxLo27jcMdII1w|489  |
|Zsdee1KEpMwjg5cqx-VWrQ|56   |
|9WuP-4jfndovEFJ4bpt5vA|62   |
|SSafXe2aUO0cXgQhEdtzrA|104  |
|4PE0tDvV9Lo8VW8kDW1SFQ|83   |
|LnFIWZM_l__4t8Qxj3pnOg|418  |
|dioXbYFdMCyE7zCK90eM3Q|133  |
|_LT9U6nhRgny5folsaTxPA|75   |
|cbY0f25GE3WLB5YUNQUgtw|74   |
+----------------------+-----+
only showing top 10 rows



In [15]:
# Filtering REVIEWS by only users identified with >= 50 REVIEWS
df_review2 = df_review1.join(df_count_reviews1, df_review1.user_id == df_count_reviews1.user_id ,"left_semi")
df_review2.count()

801411

# Baseline Algorithm

In [ ]:
# INTRODUCTION
# We predicted the rating ruf of a particular user u for a restaurant f as:
# ruf' = (µf + bu)/2 where,
# µf is the average rating of restaurant f by all users
# bu is the average rating of user u for all restaurants

In [16]:
df_review_pd = df_review2.toPandas()

In [17]:
df_avr_bus_stars = df_review_pd.groupby(['business_id']).mean()  # average rating of restaurant f by all users
df_avr_user_stars = df_review_pd.groupby(['user_id']).mean()  # average rating of user u for all restaurants

In [174]:
utility_matrix = df_avr_bus_stars.copy()

In [19]:
df_avr_bus_stars_list = df_avr_bus_stars['stars'].to_numpy()  # List of average business stars
df_avr_user_stars_list = df_avr_user_stars['stars'].to_numpy()  # List of average users stars

In [20]:
temp_list = []
for i in range(len(df_avr_user_stars_list)):
     temp_list.append((df_avr_bus_stars_list+df_avr_user_stars_list[i])/2)

In [ ]:
#utility_matrix[df_avr_user_stars.index[:]] = temp_list

# Evaluating the Baseline Algorithm

In [234]:
df_business_filtered_pd = df_business_filtered.toPandas()

In [276]:
len(df_review_pd)

801411

In [24]:
unique_users_list = df_count_reviews1.select('user_id').rdd.flatMap(lambda x: x).collect() # Getting the list of all users

In [267]:
df_review_pd_f = df_review_pd[df_review_pd['user_id'] == user]  # copying the review df to be used later
df_review_pd_f = df_review_pd_f.drop_duplicates(subset=['business_id'])  # Remove cases when user reviewed the same restaurant two times

In [281]:
# Parameters of the recommender system and evaluation
training_split = 0.8  # 80% of the dataset is used to train
k_value = 5  # Recommend five most similar items
treshold_score = 4  # Items with stars 4 or ABOVE are considered RELEVANT
num_users = 20  # Number of users evaluated
historical_precision = []  # Auxiliar variable to store all precision
cont = 1

# Running the recommender system for NUM_USERS
for user in unique_users_list[0:num_users]:
    utility_matrix = df_avr_bus_stars.copy()

    index_user = np.where(df_avr_user_stars.index[:] == user)[0].tolist()  # Find the position of the user in the avr_user_stars
    list_pred_user = temp_list[index_user[0]]  # With the position in the list, we collect the predicted values for this user in the utility matrix
    utility_matrix['pred_stars'] = list_pred_user
    utility_matrix['business_id'] = utility_matrix.index[:]

    df_review_pd_f = df_review_pd[df_review_pd['user_id'] == user]
    list_bus_user = df_review_pd_f['business_id'].to_numpy()  # List of items rated by user

    utility_matrix = utility_matrix[utility_matrix['business_id'].isin(list_bus_user)]
    utility_matrix = utility_matrix.reset_index(drop=True)
    utility_matrix = utility_matrix.drop(['stars'], axis=1)
    utility_matrix = pd.merge(utility_matrix, df_review_pd_f, on='business_id')
    utility_matrix = utility_matrix.sort_values(by=['pred_stars'], ascending=False)
    utility_matrix = pd.merge(utility_matrix, df_business_filtered_pd[['business_id','name','categories']], on='business_id')
    utility_matrix = utility_matrix.drop(['business_id','user_id'], axis = 1)
    
    
    k_pred_stars = utility_matrix['pred_stars'].to_numpy()[0:k_value]
    k_real_stars = utility_matrix['stars'].to_numpy()[0:k_value]

    precision_k = np.sum(k_real_stars>=treshold_score)/k_value # return precision score (in %)
    historical_precision.append(precision_k)
    
    print('-----------------------------------------------------------------------------------------------')
    print('Test #%i: Content-Based Recommender System for user "%s"' %(cont, user))
    cont = cont + 1
    print('Achieved precision_k with %i recommendations items (considering >=4 stars as Relevant): %f' %(k_value, precision_k))
    print('Detailed information about the recommendation list:')
    display(utility_matrix[0:k_value])
    print('\n')

print('---------------------------------------------------------------------------------------------------')
print('FINAL AVERAGE RESULTS')
print('The average performance of the system in terms of precision_k is:', np.average(historical_precision))

-----------------------------------------------------------------------------------------------
Test #1: Content-Based Recommender System for user "IeSz60ozr1yAVIH8CX9w9w"
Achieved precision_k with 5 recommendations items (considering >=4 stars as Relevant): 1.000000
Detailed information about the recommendation list:


,pred_stars,stars,name,categories
0,4.597315,5.0,Starlite Cruises,"Event Planning & Services, Restaurants, Boat C..."
1,4.565057,5.0,Pure Kitchen Organic Vegan,"Vegetarian, Juice Bars & Smoothies, Organic St..."
2,4.506406,5.0,Vampire Penguin,"Restaurants, Ice Cream & Frozen Yogurt, Shaved..."
3,4.481931,5.0,Spaddy's Coffee,"Breakfast & Brunch, Coffee & Tea, Food, Food T..."
4,4.477315,4.0,Sucre Table,"Bakeries, Custom Cakes, Food, Restaurants"




-----------------------------------------------------------------------------------------------
Test #2: Content-Based Recommender System for user "FlXBpK_YZxLo27jcMdII1w"
Achieved precision_k with 5 recommendations items (considering >=4 stars as Relevant): 1.000000
Detailed information about the recommendation list:


,pred_stars,stars,name,categories
0,4.458078,5.0,Cheers Cafe,Restaurants
1,4.458078,5.0,Little Man's Juice Bar and Grill,"Food, Juice Bars & Smoothies, Vegan, Restauran..."
2,4.458078,5.0,Little Man's Juice Bar and Grill,"Food, Juice Bars & Smoothies, Vegan, Restauran..."
3,4.458078,5.0,Tony Lukes Casa De Pasta,"Italian, Restaurants"
4,4.458078,5.0,Polka Deli Inc,"Delis, Restaurants, Sandwiches"




-----------------------------------------------------------------------------------------------
Test #3: Content-Based Recommender System for user "Zsdee1KEpMwjg5cqx-VWrQ"
Achieved precision_k with 5 recommendations items (considering >=4 stars as Relevant): 0.600000
Detailed information about the recommendation list:


...



-----------------------------------------------------------------------------------------------
Test #19: Content-Based Recommender System for user "wZWB41wmnv4pPsetuIN5Gw"
Achieved precision_k with 5 recommendations items (considering >=4 stars as Relevant): 0.800000
Detailed information about the recommendation list:


,pred_stars,stars,name,categories
0,4.420537,5.0,Dottie's Donuts,"Food, Donuts, Vegan, Restaurants"
1,4.343030,4.0,Suraya,"Active Life, Beer Gardens, Grocery, Middle Eas..."
2,4.335410,5.0,Zahav,"Nightlife, Bars, Food, Ethnic Food, Middle Eas..."
3,4.335376,5.0,Grindcore House,"Coffee & Tea, Vegetarian, Food, Restaurants, V..."
4,4.325345,3.0,The Bakeshop on 20th,"Coffee & Tea, Bakeries, Food, Soup, Breakfast ..."




-----------------------------------------------------------------------------------------------
Test #20: Content-Based Recommender System for user "rM_xdipcViCdgDPb3uKLwQ"
Achieved precision_k with 5 recommendations items (considering >=4 stars as Relevant): 1.000000
Detailed information about the recommendation list:


,pred_stars,stars,name,categories
0,3.918465,4.0,GW Fins,"Seafood, Gluten-Free, Vegetarian, Restaurants"
1,3.794486,4.0,Crown Candy Kitchen,"Specialty Food, Food, American (Traditional), ..."
2,3.772727,5.0,Monarch Restaurant & Wine Bar,"Cajun/Creole, Restaurants, Bars, American (New..."
3,3.762370,4.0,The Shaved Duck,"Barbeque, Restaurants, Soul Food, American (Ne..."
4,3.762370,4.0,The Shaved Duck,"Barbeque, Restaurants, Soul Food, American (Ne..."




---------------------------------------------------------------------------------------------------
FINAL AVERAGE RESULTS
The average performance of the system in terms of precision_k is: 0.9099999999999999
